In [6]:
import sys
import os
from autodp.calibrator_zoo import eps_delta_calibrator,generalized_eps_delta_calibrator, ana_gaussian_calibrator
from autodp import rdp_bank
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism,SubsampleGaussianMechanism, GaussianMechanism, ComposedGaussianMechanism, LaplaceMechanism
from autodp.transformer_zoo import Composition, AmplificationBySampling
%matplotlib inline 



## Example 1: calibrating noise to privacy budgets.
### Single parameter, composition is not applicable

In [2]:
calibrate = eps_delta_calibrator()
ana_calibrate = ana_gaussian_calibrator()
eps = 0.1
delta = 1e-6

mech1 = calibrate(ExactGaussianMechanism,eps,delta,[0,100],name='GM')
mech2 = ana_calibrate(ExactGaussianMechanism, eps, delta, name='Ana_GM')
print(mech1.name, mech1.params, mech1.get_approxDP(delta))
print(mech2.name, mech2.params, mech2.get_approxDP(delta))

# privacy budget is (0.1, 1e-6), calibrate noise sigma
# mech1 is for Gaussian mechanism and mech2 is for analytical Gaussian mechanism


GM {'sigma': 36.30468875627065} 0.10000000492560457
Ana_GM {'sigma': 36.304691899114694} 0.09999999565548193


## Example 2: Calibration with Gaussian mechanism under composition

In [3]:
#We now have multiple parameters --- params['coeff'] and params['sigma'].


coeff = 20
general_calibrate = generalized_eps_delta_calibrator()
params = {}
params['sigma'] = None
params['coeff'] = 20

mech3 = general_calibrate(ComposedGaussianMechanism, eps, delta, [0,1000],params=params,para_name='sigma', name='Composed_Gaussian')
print(mech3.name, mech3.params, mech3.get_approxDP(delta))
#coeff is the number of composition. The calibrator calibrates the noise with other parameters (coeff) fixed.
#[0,1000] is the range of sigma.


Composed_Gaussian {'sigma': 176.48801940301632, 'coeff': 20} 0.09999999894380397


/usr/local/lib/python3.8/site-packages/scipy/optimize/optimize.py:2522: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


## Example 3:  calibration with SubsampledGaussian mechanism

In [4]:
# We now have three parameters --- params['coeff'], params['prob'] and params['sigma'].
# The coeff and prob are fixed and the calibrator optimize over sigma. We use para_name to denote the parameter that we want to optimize over.

params['prob'] = 0.01
mech4 = general_calibrate(SubsampleGaussianMechanism, eps, delta, [0,1000],params=params,para_name='sigma', name='Subsampled_Gaussian')
print(mech4.name, mech4.params, mech4.get_approxDP(delta))


Subsampled_Gaussian {'prob': 0.01, 'sigma': 3.205947497273216, 'coeff': 20} 0.09999976859421401


## Example 4: Calibration with single parameter for Laplace mechanism

In [8]:
calibrate = generalized_eps_delta_calibrator()

eps = 0.1
delta = 1e-6
mech = calibrate(LaplaceMechanism,eps,delta,[0,100],name='Laplace')
print(mech.name, mech.params, mech.get_approxDP(delta))
#[0, 100] is the range of laplace parameter b.
# to calibrate the noise for the composed Laplace mechanism, we can define a new composed Lapalce mechanism in mechanism.zoo (similar with Example 2)

Laplace {'b': 9.99979887663798} 0.10000001127407157
